### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [69]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import re

In [36]:
url = 'https://www.booking.com/searchresults.ko.html?aid=376440&label=bdot-SIcScZhJX6z_*YtUYg62hwS267777897793%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-325272469656%3Alp1009871%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo&sid=857a41f185409d9f6d0c6b4d32c4534e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Faid%3D376440%3Blabel%3Dbdot-SIcScZhJX6z_%252AYtUYg62hwS267777897793%253Apl%253Ata%253Ap1%253Ap22%252C563%252C000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-325272469656%253Alp1009871%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo%3Bsid%3D857a41f185409d9f6d0c6b4d32c4534e%3Btmpl%3Dsearchresults%3Bcity%3D-714191%3Bclass_interval%3D1%3Bdest_id%3D-714191%3Bdest_type%3Dcity%3Bdr_ps%3DIDR%3Bdtdisc%3D0%3Bfrom_idr%3D1%3Bilp%3D1%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3De02638be83480079%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC&ssne_untouched=%EC%A0%9C%EC%A3%BC&city=-714191&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
driver = webdriver.Chrome(path)
driver.get(url)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [37]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="0bcfdaccc13c28664f10a5dcba934c42", element="e59b6069-0d26-4bd9-b805-41c19e7b2ff7")>

In [38]:
# 전체 페이지 개수 확인
pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
last_pages = ''

for idx, page in enumerate(pages):
    print(page.text)
    if idx == len(pages)-1: # 마지막 페이지
        last_pages = page.text


1
2
3
4
5
9


In [39]:
def get_curr():
    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    
    return int(curr_page.text)

In [80]:
# 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
while get_curr() <= int(last_pages):
    # scraping
    result = data_scraping()
    print(result)
    
    if get_curr() == int(last_pages):
        break
        
    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    print(curr_page.text,"페이지 완료")

    move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
    move_next.send_keys('\n')
        
    driver.implicitly_wait(1)
    time.sleep(1+random.uniform(0,1))

[('제주 아시아펜션', '3', '기존 요금 ₩25,500 현재 요금 ₩23,970'), ('제주 토비스 콘도 오션동', '', '기존 요금 ₩90,000 현재 요금 ₩84,600'), ('제주 해품달 펜션', '3', '기존 요금 ₩300,296 현재 요금 ₩225,600'), ('일성 제주콘도미디엄', '', '기존 요금 ₩98,000 현재 요금 ₩92,120'), ('제주 오름펜션', '3', '기존 요금 ₩49,400 현재 요금 ₩46,436'), ('동복리671', '3', '기존 요금 ₩100,263 현재 요금 ₩56,400'), ('부띠크 제주민박 살롱 드 탱자', '3', '기존 요금 ₩75,000 현재 요금 ₩70,500'), ('하버 호텔', '3', '₩54,159'), ('선양 빌리지 펜션', '3', '기존 요금 ₩220,000 현재 요금 ₩206,800'), ('애월바다호텔 제주', '3', '₩57,487'), ('', '', ''), ('메가 리조트', '', '기존 요금 ₩60,000 현재 요금 ₩56,400'), ('허브인펜션', '3', '기존 요금 ₩49,900 현재 요금 ₩46,906'), ('제주 삼다 호스텔', '', '₩37,476'), ('심스 호텔', '', '₩44,935'), ('쉬멍놀멍 게스트하우스 & 펜션', '', '요금 ₩75,000'), ('Milgam Central Hotel', '3', '기존 요금 ₩28,000 현재 요금 ₩26,320'), ('노블레스 호텔', '3', '요금 ₩60,000'), ('제주 또올레 펜션', '3', '기존 요금 ₩80,000 현재 요금 ₩75,200'), ('메이저 호텔 2', '2', '요금 ₩60,000'), ('농부와 돌집펜션', '3', '기존 요금 ₩249,595 현재 요금 ₩159,800'), ('비자림 리조트', '', '기존 요금 ₩80,211 현재 요금 ₩45,120'), ('라움 호텔', '3', '요금 ₩45,000'), ('비스타 리조트 제주

In [40]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [67]:
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            tmp_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if tmp_price.text == '':
                tmp_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(tmp_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [57]:
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [77]:
def data_scraping():
    res_stars = []
    res_names = []
    res_prices = []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    
    res_prices = get_prices(listed)
              
    return list(zip(res_names, res_stars, res_prices))

data_scraping()

[('밀레니엄 호텔', '', '기존 요금 ₩31,700 현재 요금 ₩29,798'),
 ('메이저 호텔', '2', '기존 요금 ₩60,000 현재 요금 ₩52,800'),
 ('낭만제주 펜션', '', ''),
 ('퓨어스테이', '', ''),
 ('이안 게스트하우스', '', '요금 ₩50,000'),
 ('거문오름 호스텔', '', '요금 ₩54,000'),
 ('엠제이호텔', '', '₩58,596'),
 ('J-ONE Hotel 제이원호텔', '2', '요금 ₩48,000'),
 ('제주 나인 리조트', '2', '기존 요금 ₩83,000 현재 요금 ₩73,040'),
 ('제주 바다성 펜션', '3', '요금 ₩35,000'),
 ('', '', ''),
 ('Helen Jeju Swiss town', '3', '기존 요금 ₩67,100 현재 요금 ₩63,074'),
 ('대웅 황토리조트', '', '기존 요금 ₩150,091 현재 요금 ₩73,320'),
 ('아뜨네 펜션', '3', '₩77,317'),
 ('까사린다', '', '기존 요금 ₩150,000 현재 요금 ₩141,000'),
 ('숨 게스트하우스 제주공항', '', '요금 ₩43,800'),
 ('노마드인제주 게스트하우스', '', '요금 ₩50,000'),
 ('예스준 게스트하우스', '', '요금 ₩50,000'),
 ('제주도 하숙집', '', '기존 요금 ₩44,000 현재 요금 ₩41,360')]

---

In [1]:
# 한글 설정
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

f_path = "/Users/seyoung/Library/Fonts/AppleGothic.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)